# Assignment 2 - Part B: Building a decision tree

This is a skeleton of a decision tree classifier for the example data set in `data/example.csv`.

In [32]:
import csv
import math
from statistics import median, mode
from collections import Counter
from enum import Enum

Some simple type definitions.

In [33]:
class AttrType(Enum):
    cat = 0  # categorical (qualitative) attribute
    num = 1  # numerical (quantitative) attribute
    target = 2  # target label


class NodeType(Enum):
    root = 0
    internal = 1
    leaf = 2


class SplitType(Enum):
    bin = 0  # binary split
    multi = 1  # multi-way split

Also, some basic classes to represent an attribute, a spltting procedure, and a node.

In [34]:
class Attribute(object):
    def __init__(self, label, type):
        assert type in AttrType
        self.label = label
        self.type = type
        self.stat = None  # holds mean for numerical and mode for categorical attributes


class Splitting(object):
    def __init__(self, attr, infogain, split_type, cond, splits):
        self.attr = attr  # attribute ID (index in ATTR)
        self.infogain = infogain  # information gain if splitting is done on this attribute
        self.split_type = split_type  # one of SplitType
        self.cond = cond  # splitting condition, i.e., values on outgoing edges
        self.splits = splits  # list of training records (IDs) for each slitting condition


class Node(object):
    def __init__(self, id, type, parent_id, children=None, edge_value=None, val=None, split_type=None, split_cond=None,
                 infogain=None):
        self.id = id  # ID (same as the index in DT.model list)
        self.type = type  # one of NodeType
        self.parent_id = parent_id  # ID of parent node (None if root)
        self.children = children  # list of IDs of child nodes
        self.edge_value = edge_value  # the value of the incoming edge (only if not root node)
        self.val = val  # if root or internal node: the attribute that is compared at that node; if leaf node: the target value
        self.split_type = split_type  # one of SplitType
        self.split_cond = split_cond  # splitting condition (median value for binary splits on numerical values; otherwise a list of categorical values (corresponding to child nodes))
        self.infogain = infogain

    def append_child(self, node_id):
        self.children.append(node_id)

The input filename is hard-coded.

In [35]:
INFILE = "data/basketball.train.csv"
TESTFILE = "data/basketball.test.csv"
OUTPUT_FILE = "data/dt_basketball.pred.csv"


def load_test_data():
        with open(TESTFILE) as csvfile:
            data = []
            csvreader = csv.reader(csvfile, delimiter=',')
            for row in csvreader:
                rec = []
                for i in range(len(ATTR) - 1):
                    val = row[i].strip()
                    # convert numerical attributes
                    if ATTR[i].type == AttrType.num:  # Note that this will break for "?" (missing attribute)
                        val = float(val)
                    rec.append(val)
                data.append(rec)
        return data

The attribute labels types are hard-coded too (the same order as in the file!).

In [36]:
ATTR = [Attribute("LOCATION", AttrType.cat), Attribute("W", AttrType.cat),
        Attribute("FINAL_MARGIN", AttrType.num), Attribute("SHOT_NUMBER", AttrType.num), 
        Attribute("PERIOD", AttrType.cat), Attribute("GAME_CLOCK", AttrType.num), 
        Attribute("SHOT_CLOCK", AttrType.num), Attribute("DRIBBLES", AttrType.num), 
        Attribute("TOUCH_TIME", AttrType.num), Attribute("SHOT_DIST", AttrType.num),
        Attribute("PTS_TYPE", AttrType.cat), Attribute("CLOSE_DEF_DIST", AttrType.num),
        Attribute("Target", AttrType.target)]

The index of the target attribute (assuming it's the last).

In [37]:
IDX_TARGET = len(ATTR) - 1 

A main class DT representing the decision tree classifier. It could represent with methods:

  - a given inpurity measure;
  - the search for the best attribute to split with;
  - the addition of a node to the tree;
  - a convenient model printer;
  - the recursive call for obtaining a tree;
  - a builder and an applier.

In [38]:
class DT(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.model = None  # decision tree model
        self.default_class = None  # default target class

    def __load_data(self):
        with open(INFILE) as csvfile:
            self.data = []
            csvreader = csv.reader(csvfile, delimiter=',')
            for row in csvreader:
                rec = []
                for i in range(len(ATTR)):
                    val = row[i].strip()
                    # convert numerical attributes
                    if ATTR[i].type == AttrType.num:  # Note that this will break for "?" (missing attribute)
                        val = float(val)
                    rec.append(val)
                # print(rec)
                self.data.append(rec)
                # self.data.append([element.strip() for element in row])  # strip spaces
                
    
    def __median(self, attrNum):
        attrList = []
        
        for i in range(0, len(self.data)):
            attrList.append(self.data[i][attrNum])
            
        return median(attrList)
        

    def __entropy(self, records):
        """
        Calculates entropy for a selection of records.

        :param records: Data records (given by indices)
        """
        yes = 0
        no = 0

        for r in records:
            if (self.data[r][IDX_TARGET] == "made"):
                yes += 1
            if (self.data[r][IDX_TARGET] == "missed"):
                no += 1

        entropy = -no/(no+yes)*math.log2(no/(no+yes)) - yes/(no+yes)*math.log2(yes/(no+yes))

        return entropy

    
    def __find_best_attr(self, attrs, records):
        """
        Finds the attribute with the largest gain.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :return:
        """
        entropy_p = self.__entropy(records)  # parent's entropy
        splittings = []  # holds the splitting information for each attribute

        for a in attrs:
            assert ATTR[a].type in AttrType
            splits = {}  # record IDs corresponding to each split
            split_mode = None
            split_cond = None
            # splitting condition depends on the attribute type
            if ATTR[a].type == AttrType.target:  # skip target attribute
                continue
            elif ATTR[a].type == AttrType.cat:  # categorical attribute
                # multi-way split on each possible value
                split_mode = SplitType.multi
                # each possible attr value corresponds to a split (indexed with categorical labels)
                # Note: it's important to consider attr values from the entire training set
                split_cond = set([self.data[idx][a] for idx in range(len(self.data))])
                
                # `splits[val]` holds a list of records for a given split,
                # where `val` is an element of `split_cond`
                for e in split_cond:
                    splits[e] = []
                    for r in records:
                        if self.data[r][a] == e:
                            splits[e].append(r)
                
            elif ATTR[a].type == AttrType.num:  # numerical attribute => binary split on median value
                split_mode = SplitType.bin
                split_cond = self.__median(a)  # (i.e., if less or equal than this value)
                        
                greater = []
                less_equal = []

                for r in records:
                    if(self.data[r][a] > split_cond):
                        greater.append(r)
                    elif (self.data[r][a] <= split_cond):
                        less_equal.append(r)

                str = "<=" + repr(split_cond)
                splits[str] = less_equal
                str = ">" + repr(split_cond)
                splits[str] = greater

            # compute gain for attribute a
            infogain = entropy_p

            for split in splits:
                yes = 0
                no = 0
                
                for r in splits[split]:
                    if (self.data[r][IDX_TARGET] == "made"):
                         yes += 1
                    if (self.data[r][IDX_TARGET] == "missed"):
                        no += 1        
                
                if(no == 0 and yes == 0):
                    entropy = 1
                elif(no == 0 or yes == 0):
                    entropy = 0
                else:
                    entropy = -no / (no + yes) * math.log2(no / (no + yes)) - yes / (no + yes) * math.log2(yes / (no + yes))
                
                infogain -= ((yes + no) / len(records)) * entropy
                
            print("Information Gain for attribute {0}: {1}".format(ATTR[a].label, infogain))

            splitting = Splitting(a, infogain, split_mode, split_cond, splits)
            splittings.append(splitting)

        # find best splitting
        best_splitting = sorted(splittings, key=lambda x: x.infogain, reverse=True)[0]
        return best_splitting

    def __add_node(self, parent_id, node_type=NodeType.internal, edge_value=None, val=None, split_type=None,
                   split_cond=None):
        """
        Adds a node to the decision tree.

        :param parent_id:
        :param node_type:
        :param edge_value:
        :param val:
        :param split_type:
        :param split_cond:
        :return:
        """
        node_id = len(self.model)  # id of the newly assigned node
        if not self.model:  # the tree is empty
            node_type = NodeType.root

        node = Node(node_id, node_type, parent_id, children=[], edge_value=edge_value, val=val, split_type=split_type,
                    split_cond=split_cond)
        self.model.append(node)

        # also add it as a child of the parent node
        if parent_id is not None:
            self.model[parent_id].append_child(node_id)

        return node_id

    def __id3(self, attrs, records, parent_id=None, value=None):
        """
        Function ID3 that returns a decision tree.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :param parent_id: ID of parent node
        :param value: Value corresponding to the parent attribute, i.e., label of the edge on which we arrived to this node
        :return:
        """
        # empty training set or empty set of attributes => create leaf node with default class
        if not records or not attrs:
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=self.default_class)
            return

        # if all records have the same target value => create leaf node with that target value
        same = all(self.data[idx][IDX_TARGET] == self.data[records[0]][IDX_TARGET] for idx in records)
        if same:
            target = self.data[records[0]][IDX_TARGET]
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=target)
            return
        
        ###############################################################################################
        # find majority class
        if len(records) < 500:
            classes = []
            
            for r in records:
                classes.append(self.data[r][IDX_TARGET])

            try:
                majorityClass = statistics.mode(classes)
            except:
                print("no mode")
                majorityClass = self.default_class

            if(majorityClass  == TARGET_ATTR_SUCCESS):
                target = TARGET_ATTR_SUCCESS
            elif(majorityClass == TARGET_ATTR_FAILURE):
                target = TARGET_ATTR_FAILURE

            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=target)
            return
        ###############################################################################################

        # find the attribute with the largest gain
        splitting = self.__find_best_attr(attrs, records)
        # add node
        node_id = self.__add_node(parent_id, edge_value=value, val=splitting.attr, 
                                  split_type=splitting.split_type, split_cond=splitting.cond)
        
        print('\nWorking . . .')

        newattrs = attrs.copy()
        newattrs.remove(splitting.attr)

        if(splitting.split_type == SplitType.bin):
            less_equal = []
            greater = []
            for r in records:
                if (self.data[r][splitting.attr] > splitting.cond):
                    greater.append(r)
                elif (self.data[r][splitting.attr] <= splitting.cond):
                    less_equal.append(r)
            self.__id3(newattrs, greater, parent_id=node_id, value=False)
            self.__id3(newattrs, less_equal, parent_id=node_id, value=True)

        if(splitting.split_type == SplitType.multi):
            for sc in splitting.cond:
                subset = []
                for r in records:
                    if(self.data[r][splitting.attr] == sc):
                        subset.append(r)
                self.__id3(newattrs, subset, parent_id=node_id, value=sc)
                

    def print_model(self, node_id=0, level=0):
        node = self.model[node_id]
        indent = "  " * level
        if node.type == NodeType.leaf:
            print(indent + str(node.edge_value) + " [Leaf node] class=" + node.val)
        else:
            cond = " <= " + str(node.split_cond) if ATTR[node.val].type == AttrType.num else " == ? "
            if node.type == NodeType.root:
                print("[Root node] '" + ATTR[node.val].label + "'" + cond)
            else:
                print(indent + str(node.edge_value) + " [Internal node] '" + ATTR[node.val].label + "'" + cond)
            # print tree for child notes recursively
            for n_id in node.children:
                self.print_model(n_id, level + 1)

    def build_model(self):
        self.__load_data()
        self.model = []  # holds the decision tree model, represented as a list of nodes
        # Get majority class
        #   Note: Counter returns a dictionary, most_common(x) returns a list with the x most common elements as
        #         (key, count) tuples; we need to take the first element of the list and the first element of the tuple
        self.default_class = Counter([x[IDX_TARGET] for x in self.data]).most_common(1)[0][0]
        self.__id3(set(range(len(ATTR) - 1)), list(range(len(self.data))))

    def apply_model(self, record):
        node = self.model[0]
        while node.type != NodeType.leaf:
            # TODO based on the value of the record's attribute that is tested in `node`,
            # set `node` to one of its child nodes until a leaf node is reached
            attr_val = record[node.val]

            if ATTR[node.val].type == AttrType.num:
                if record[node.val] > node.split_cond:
                    edge = False
                elif record[node.val] <= node.split_cond:
                    edge = True
            else:
                edge = attr_val

            for child_id in node.children:
                child_edge_value = self.model[child_id].edge_value

                if self.model[child_id].edge_value == edge:
                    node = self.model[child_id]
                    
        return node.val

Finally, the main function building a decision tree model, printing it and applying it on some unseen records.

In [39]:
def main():
    dt = DT()
    print("Build model:")
    dt.build_model()
    dt.print_model()
    
    test = load_test_data()

    print("\nApply model:")
    with open(OUTPUT_FILE, 'w') as output:
        output.write("Id,Target\n")
        for id, data in enumerate(test):
            res = dt.apply_model(data)
            output.write("%s,%s\n" % (id + 1, res))
    print("Results written to {0}".format(OUTPUT_FILE))


if __name__ == "__main__":
    main()

Build model:
Information Gain for attribute LOCATION: 0.0001403197935238576
Information Gain for attribute W: 0.0015119418826527453
Information Gain for attribute FINAL_MARGIN: 0.0013424812401782171
Information Gain for attribute SHOT_NUMBER: 0.00010174272372082127
Information Gain for attribute PERIOD: 0.00026749841779211136
Information Gain for attribute GAME_CLOCK: 1.882332878455628e-08
Information Gain for attribute SHOT_CLOCK: 0.003105048901265617
Information Gain for attribute DRIBBLES: 0.0014535159128711284
Information Gain for attribute TOUCH_TIME: 0.002509204053641023
Information Gain for attribute SHOT_DIST: 0.017133483068884237
Information Gain for attribute PTS_TYPE: 0.01102213028657592
Information Gain for attribute CLOSE_DEF_DIST: 0.00042678007200808166

Working . . .
Information Gain for attribute LOCATION: 0.00015536978745817054
Information Gain for attribute W: 0.0020081897525431325
Information Gain for attribute FINAL_MARGIN: 0.0018171489324245171
Information Gain for

Information Gain for attribute DRIBBLES: 0.005478729494017953

Working . . .
Information Gain for attribute W: 0.0
Information Gain for attribute DRIBBLES: 0.009693668617616447

Working . . .
Information Gain for attribute W: 0.0

Working . . .
Information Gain for attribute W: 0.0

Working . . .
Information Gain for attribute W: 0.0
Information Gain for attribute DRIBBLES: 0.0023146583577101643

Working . . .
Information Gain for attribute W: 0.0

Working . . .
Information Gain for attribute W: 0.0

Working . . .
Information Gain for attribute W: 1.9972198339601732e-08
Information Gain for attribute SHOT_NUMBER: 0.00037878129799839844
Information Gain for attribute GAME_CLOCK: 0.00035482108370943344
Information Gain for attribute SHOT_CLOCK: 9.625612519492677e-06
Information Gain for attribute DRIBBLES: 6.022057836729822e-06

Working . . .
Information Gain for attribute W: 0.0008179267247211408
Information Gain for attribute GAME_CLOCK: 4.08187747096278e-05
Information Gain for attrib

Information Gain for attribute SHOT_CLOCK: 0.0004968417863503993
Information Gain for attribute DRIBBLES: 0.00011195799636387616

Working . . .
Information Gain for attribute W: 0.0011797757390772379
Information Gain for attribute FINAL_MARGIN: 0.0006346265424771458
Information Gain for attribute SHOT_CLOCK: 0.0009486775042510942
Information Gain for attribute DRIBBLES: 4.467695802679028e-05

Working . . .
Information Gain for attribute FINAL_MARGIN: 0.0011120814820214653
Information Gain for attribute SHOT_CLOCK: 0.00016346761354679717
Information Gain for attribute DRIBBLES: 0.0

Working . . .
Information Gain for attribute SHOT_CLOCK: 3.166632805590153e-05
Information Gain for attribute DRIBBLES: 0.0007084134007978582

Working . . .
Information Gain for attribute SHOT_CLOCK: 0.0001441459272368273

Working . . .
Information Gain for attribute SHOT_CLOCK: 0.0013397424044413464

Working . . .
Information Gain for attribute SHOT_CLOCK: 0.31127812445913283
Information Gain for attribute 

Information Gain for attribute DRIBBLES: 0.002657772941713765

Working . . .
Information Gain for attribute LOCATION: 2.9772828728624745e-05
Information Gain for attribute FINAL_MARGIN: 0.00010672850460369254
Information Gain for attribute SHOT_NUMBER: 0.0009094337813642522
Information Gain for attribute GAME_CLOCK: 8.919137221541362e-05
Information Gain for attribute SHOT_CLOCK: 0.0033348028881578506

Working . . .
Information Gain for attribute LOCATION: 0.00028394986708302206
Information Gain for attribute FINAL_MARGIN: 0.001705140648852077
Information Gain for attribute SHOT_NUMBER: 0.00021893688837509168
Information Gain for attribute GAME_CLOCK: 7.680821133448923e-05

Working . . .
Information Gain for attribute LOCATION: 0.0004058600981231564
Information Gain for attribute SHOT_NUMBER: -4.163336342344337e-17
Information Gain for attribute GAME_CLOCK: 0.00023123903879684882

Working . . .
Information Gain for attribute SHOT_NUMBER: 0.00023277986497310943
Information Gain for attr

Information Gain for attribute LOCATION: 0.005617597224257698
Information Gain for attribute FINAL_MARGIN: 0.0

Working . . .
Information Gain for attribute FINAL_MARGIN: 0.0

Working . . .
Information Gain for attribute FINAL_MARGIN: 0.0

Working . . .
Information Gain for attribute LOCATION: 0.04812703040826949
Information Gain for attribute FINAL_MARGIN: 0.0
Information Gain for attribute GAME_CLOCK: 0.06062407338396347
Information Gain for attribute SHOT_CLOCK: 0.011265848648557342

Working . . .
Information Gain for attribute LOCATION: 0.11036014405977657
Information Gain for attribute FINAL_MARGIN: 0.0
Information Gain for attribute SHOT_CLOCK: 0.03494091395622845

Working . . .
Information Gain for attribute FINAL_MARGIN: 0.0
Information Gain for attribute SHOT_CLOCK: 0.07278022578373267

Working . . .
Information Gain for attribute FINAL_MARGIN: 0.0

Working . . .
Information Gain for attribute FINAL_MARGIN: 0.0

Working . . .
Information Gain for attribute FINAL_MARGIN: 0.0
In

...

                        False [Leaf node] class=missed
                        True [Leaf node] class=missed
                    True [Leaf node] class=missed
                  H [Internal node] 'FINAL_MARGIN' <= 2.0
                    False [Internal node] 'GAME_CLOCK' <= 363.0
                      False [Internal node] 'SHOT_CLOCK' <= 12.4
                        False [Leaf node] class=missed
                        True [Leaf node] class=missed
                      True [Internal node] 'SHOT_CLOCK' <= 12.4
                        False [Leaf node] class=missed
                        True [Leaf node] class=missed
                    True [Leaf node] class=made
              True [Internal node] 'FINAL_MARGIN' <= 2.0
                False [Internal node] 'LOCATION' == ? 
                  A [Internal node] 'SHOT_NUMBER' <= 5.0
                    False [Internal node] 'GAME_CLOCK' <= 363.0
                      False [Leaf node] class=missed
                      True [Internal n

                  False [Leaf node] class=missed
                  True [Internal node] 'TOUCH_TIME' <= 1.5
                    False [Internal node] 'GAME_CLOCK' <= 363.0
                      False [Leaf node] class=missed
                      True [Internal node] 'DRIBBLES' <= 0.0
                        False [Leaf node] class=missed
                        True [Leaf node] class=missed
                    True [Internal node] 'GAME_CLOCK' <= 363.0
                      False [Leaf node] class=missed
                      True [Internal node] 'DRIBBLES' <= 0.0
                        False [Leaf node] class=missed
                        True [Leaf node] class=missed
                H [Internal node] 'DRIBBLES' <= 0.0
                  False [Internal node] 'TOUCH_TIME' <= 1.5
                    False [Internal node] 'FINAL_MARGIN' <= 2.0
                      False [Leaf node] class=missed
                      True [Internal node] 'GAME_CLOCK' <= 363.0
                        F

                    True [Leaf node] class=made
                  H [Internal node] 'DRIBBLES' <= 0.0
                    False [Internal node] 'SHOT_NUMBER' <= 5.0
                      False [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                      True [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                    True [Internal node] 'SHOT_NUMBER' <= 5.0
                      False [Leaf node] class=missed
                      True [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
              True [Internal node] 'FINAL_MARGIN' <= 2.0
                False [Internal node] 'LOCATION' == ? 
                  A [Internal node] 'DRIBBLES' <= 0.0
                    False [Internal node] 'SHOT_NUMBER' <= 5.0
    

                  False [Internal node] 'LOCATION' == ? 
                    A [Internal node] 'W' == ? 
                      W [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                      L [Leaf node] class=missed
                    H [Internal node] 'W' == ? 
                      W [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                      L [Leaf node] class=missed
                  True [Internal node] 'W' == ? 
                    W [Leaf node] class=made
                    L [Internal node] 'LOCATION' == ? 
                      A [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                      H [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
               

                        3 [Leaf node] class=missed
                      L [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                  H [Internal node] 'W' == ? 
                    W [Internal node] 'FINAL_MARGIN' <= 2.0
                      False [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                      True [Leaf node] class=made
                    L [Internal node] 'FINAL_MARGIN' <= 2.0
                      False [Leaf node] class=missed
                      True [Internal node] 'PTS_TYPE' == ? 
                        2 [Leaf node] class=missed
                        3 [Leaf node] class=missed
                True [Internal node] 'FINAL_MARGIN' <= 2.0
                  False [Internal node] 'LOCATION' == ? 
                    A [Internal node] 'W' == ? 
                      W [Interna